In [1]:
%load_ext autoreload
%autoreload 2
import itertools
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))
PATH = 's3://chess-puzzles/single-best-mate/{year}-{month:0>2}/{shard}.csv'


def get_df(years, months):
    dfs = []
    for year in years:
        for month in months:
            for shard in itertools.count():
                path = PATH.format(year=year, month=month, shard=shard)
                try:
                    df = pd.read_csv(path)
                    print(path)
                except Exception as e:
                    print(e)
                    break

                df = df.dropna(how='all')
                dfs.append(df)
    return pd.concat(dfs)


# df = get_df([2015], [11, 12])
# df.to_csv('single_best_mate_2015_12.csv', index=False)
df = pd.read_csv('single_best_mate_2015_12.csv')
print(len(df))

6967


In [33]:
import features
feature_df = features.get_feature_df(df, [features.MovesFeatures], ['fen', 'best_move', 'best_mate', 'best_score2', 'correct'])
print(len(df))
feature_df[:1]


6967


,number_of_bishop_moves,number_of_captures,number_of_checks,number_of_knight_moves,number_of_moves,number_of_pawn_moves,number_of_queen_moves,number_of_rook_moves,fen,best_move,best_mate,best_score2,correct
0,8,5,5,0,40,7,12,10,r4rk1/2p2p2/1p1p1p2/p3p3/P1P2qPp/5b2/1P2B2P/3R...,f4g4,3.0,1879.0,True


# Boards

In [11]:
import utils

queries = [
    # TODO: ignore boards where best_pv involves promotion.    
    
#     'best_score2 > -2000',
    'best_score2 > 4000',
    
#     'best_mate == 1',
#     'pv0_piece_type == 5',
    #     'their_king_is_back_rank == True'
#     'pv0_piece_type == 4',
]

filtered_df = utils.filter_df(feature_df, queries)
html = utils.df_to_table(filtered_df[:10], ['best_mate'])
HTML(html)

# Plots

In [34]:
# TODO: combine this with boards?
# TODO: make this a single cell in a notebook that is our vis tool.
import plotly.express as px
from plotly.offline import iplot
import ipywidgets as widgets
from ipywidgets import interact


# TODO: how to do filtering?
# TODO: what other graphing functionality do we want?
def f(x):
    fig = px.histogram(feature_df, x=x, color='correct')
    iplot(fig)


# TODO: how to get all features? Use get features func on classes?
interact(f, x=['number_of_moves', 'number_of_checks', 'number_of_captures']);

interactive(children=(Dropdown(description='x', options=('number_of_moves', 'number_of_checks', 'number_of_cap…

# Models

In [ ]:
import features
feature_df = features.get_feature_df(df, [features.Features, features.MovesFeatures, features.BoardFeatures, features.MovesFeatures, features.MateIn1Features])
feature_df[:1]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

x = feature_df.values
y = df[:len(x)]['correct']

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)
s = StandardScaler()
x_train = s.fit_transform(x_train)
x_test = s.transform(x_test)

models = [
    DummyClassifier(strategy='most_frequent'),
    LogisticRegression(),
]

for model in models:
    score = model.fit(x_train, y_train).score(x_test, y_test)
    print('{:.0%}'.format(score))